In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [31]:
training_data = datasets.MNIST(
	root="data",
	train=True,
	download=True,
 	transform=ToTensor()	
)

testing_data = datasets.MNIST(
	root="data",
	train=False,
	download=True,
	transform=ToTensor()
)

In [3]:
batch_size = 128
training_data = DataLoader(
	training_data,
	batch_size=batch_size
)	

testing_data = DataLoader(
	testing_data,
	batch_size=batch_size
)

In [6]:
for X, y in training_data:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]), torch.int64


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
class NeuralNetworkMNIST(nn.Module):
	def __init__(self):
		super(NeuralNetworkMNIST, self).__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack =nn.Sequential(
			nn.Linear(28*28, 1024),
			nn.ReLU(),
			nn.Linear(1024, 512),		
			nn.ReLU(),
			nn.Linear(512, 10),
		)
	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetworkMNIST().to(device)
print(model)
	

NeuralNetworkMNIST(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
# 1. Set the model to training mode: This enables dropout and batch normalization layers.
	size = len(dataloader.dataset)
	model.train()
# 2. Iterate over the dataloader: Loop through batches of data.
	for batch, (X, y) in enumerate(dataloader):
		X, y = X.to(device), y.to(device)
# 3. Zero the gradients: Clear previous gradients to prevent accumulation.
		optimizer.zero_grad()
# 4. Forward pass: Pass the input data through the model to get predictions.
		pred = model(X)
# 5. Compute the loss: Use the loss function to compare predictions with true labels.
		loss = loss_fn(pred, y)
# 6. Backward pass: Calculate gradients with respect to the loss.
		loss.backward()
# 7. Update weights: Use the optimizer to adjust model parameters based on gradients.
		optimizer.step()
		if batch % 100 == 0:
			loss, current = loss.item(), batch * len(X)
			print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
def test(dataloader, model, loss_fn):
	size = len(dataloader.dataset)
	model.eval()
	total_loss, correct = 0, 0
	with torch.no_grad():
		for batch, (X, y) in enumerate(dataloader):
			X, y = X.to(device), y.to(device)
			pred = model(X)
			total_loss += loss_fn(pred, y).item()
			correct += (pred.argmax(1) == y).type(torch.float).sum().item()
	total_loss/= size
	correct /= size
	print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {total_loss:>8f}\n")


In [27]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_data, model, loss_fn, optimizer)
    test(testing_data, model, loss_fn)

Epoch 1
-------------------------------
loss: 0.076017  [    0/60000]
loss: 0.087619  [12800/60000]
loss: 0.107014  [25600/60000]
loss: 0.066440  [38400/60000]
loss: 0.110896  [51200/60000]
Test error: 
 Accuracy: 95.8%, Avg loss: 0.001045

Epoch 2
-------------------------------
loss: 0.098164  [    0/60000]
loss: 0.067306  [12800/60000]
loss: 0.095153  [25600/60000]
loss: 0.048946  [38400/60000]
loss: 0.105829  [51200/60000]
Test error: 
 Accuracy: 95.3%, Avg loss: 0.001280

Epoch 3
-------------------------------
loss: 0.044147  [    0/60000]
loss: 0.046111  [12800/60000]
loss: 0.047516  [25600/60000]
loss: 0.022430  [38400/60000]
loss: 0.119219  [51200/60000]
Test error: 
 Accuracy: 97.0%, Avg loss: 0.000811

Epoch 4
-------------------------------
loss: 0.075996  [    0/60000]
loss: 0.091946  [12800/60000]
loss: 0.054574  [25600/60000]
loss: 0.026287  [38400/60000]
loss: 0.089676  [51200/60000]
Test error: 
 Accuracy: 96.7%, Avg loss: 0.001019

Epoch 5
----------------------------

In [28]:
torch.save(model.state_dict(), "MNISTmodel.pth")
print("Saved PyTorch Model State to MNISTmodel.pth")

Saved PyTorch Model State to MNISTmodel.pth


In [29]:
model = NeuralNetworkMNIST().to(device)
model.load_state_dict(torch.load("MNISTmodel.pth", weights_only=True))

<All keys matched successfully>

In [35]:
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

model.eval()
x, y = testing_data[1][0], testing_data[1][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "2", Actual: "2"
